## BlockEncoding class showcase

motivation

### how to construct a BE object (quantum numpy)

#### ``from_operator``
Explain that it block‑encodes the Hermitian part of the operator and internally uses a Pauli decomposition.​

In [ ]:
###### EXAMPLE

#### ``.from_array``
Square matrix, size a power of two, supports `ndarray`, `csrarray`, `csrmatrix`.​

In [ ]:
### EXAMPLE

#### ``.from_LCU``
Introduce `BlockEncoding.from_lcu(coeffs, unitaries)` and the PREP–SEL–PREP† construction.​

Explain padding of coefficients, normalization α=∑i∣αi∣α=∑i∣αi∣, sign handling via additional phase flips, and how `qswitch` is used inside. ​

In [ ]:
### EXAMPLE

### EXAMPLE: Laplacian example already in the doc (1D discrete Laplace), show how to write `unitaries = [I, V, V_dag]` and `coeffs`, then `BE = BlockEncoding.from_lcu(coeffs, unitaries)`.​


#### Chebyshev polynomial using ``.chebyshev``

EXPLAIN

In [ ]:
### EXAMPLE

#### Inverse using ``.inv``

EXPLAIN

In [ ]:
### EXAMPLE

#### Polynomial transformations using ``.poly``

EXPLAIN

In [ ]:
### EXAMPLE

#### Hamiltnian simulation using ``.sim``

EXPLAIN

In [ ]:
### EXAMPLE